In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
size(d) # Should be 504x8

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/dev/Turing/src/Turing.jl:24


(504, 8)

pulled_left, condition, prosoc_left

In [2]:
@model m10_3(y, x₁, x₂) = begin
    α ~ Normal(0, 10)
    βp ~ Normal(0, 10)
    βpC ~ Normal(0, 10)

    for i ∈ 1:length(y)
        p = logistic(α + (βp + βpC * x₁[i]) * x₂[i])
        y[i] ~ Binomial(1, p)
    end
end;

posterior = sample(m10_3(d[:,:pulled_left], d[:,:condition], d[:,:prosoc_left]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.2
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.07401522714345364, std = [1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 297.0978411399998;
  #lf / sample        = 0.0;
  #evals / sample     = 28.515;
  pre-cond. metric    = [1.0, 1.0, 1.0].


Fix the inclusion of adaptation samples

In [3]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

Rethinking result

In [4]:
m_10_03t_result = "
      Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a    0.05   0.13      -0.15       0.25  3284    1
 bp   0.62   0.22       0.28       0.98  3032    1
 bpC -0.11   0.26      -0.53       0.29  3184    1
";

Describe the draws

In [5]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                   SD                       Naive SE              MCSE         ESS   
       α    0.052410534  0.126685288268529122612804 0.0040061405696356086961196 0.0053737770  555.76742
     βpC   -0.108159213  0.271545412461579149798752 0.0085870199154846008765540 0.0150758411  324.43062
  lf_num    0.000000000  0.000000000000000000000000 0.0000000000000000000000000 0.0000000000        NaN
 elapsed    0.135806777  0.076395598638849773753812 0.0024158409491082449491173 0.0028992753  694.31694
 epsilon    0.074015227  0.000000000000000097193123 0.0000000000000000030735164 0.0000000000 1000.00000
      βp    0.608133222  0.224832286090436761494260 0.0071098211558837404872024 0.0129030657  303.62081
      lp -349.399056989  1.277684537767762806836913 0.0404039327052555866104377 0.0754142472  287.03872
eval_num   25.720000000 13.748841065000011241181710 0.4347

End of m_10_03t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*